# Install requirements through pip

In [25]:
!pip install --user tensorflow==2.3.1
!pip install --user nltk==3.5
!pip install --user colorama==0.4.3
!pip install --user numpy==1.18.5
!pip install --user scikit_learn==0.23.2
!pip install --user Flask==1.1.2
!pip install -U mapbox
!pip install --user requests

ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.1 (from versions: 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.7.0rc0, 2.7.0rc1, 2.7.0)
ERROR: No matching distribution found for tensorflow==2.3.1


    ERROR: Command errored out with exit status 1:
     command: 'D:\Anaconda\python.exe' 'D:\Anaconda\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' prepare_metadata_for_build_wheel 'C:\Users\Mels\AppData\Local\Temp\tmpv2qzysa8'
         cwd: C:\Users\Mels\AppData\Local\Temp\pip-install-ysjh4ykq\numpy_416e10a1a1a94fafbce5eee587ee545f
    Complete output (241 lines):
    Running from numpy source directory.
    setup.py:461: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
      run_build = parse_setuppy_commands()
    Processing numpy/random\_bounded_integers.pxd.in
    Processing numpy/random\mtrand.pyx
    Processing numpy/random\_bit_generator.pyx
    Processing numpy/random\_bounded_integers.pyx.in
    Processing numpy/random\_common.pyx
    Processing numpy/random\_generator.pyx
    Processing numpy/random\_mt19937.pyx
    Processing numpy/random\_pcg64.pyx
    Processing numpy/random\_philox.pyx
    


  Using cached numpy-1.18.5.zip (5.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'error'
  Using cached scikit-learn-0.23.2.tar.gz (7.2 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'D:\Anaconda\python.exe' 'C:\Users\Mels\AppData\Local\Temp\pip-standalone-pip-qsunw332\__env_pip__.zip\pip' install --ignore-installed --no-user --prefix 'C:\Users\Mels\AppData\Local\Temp\pip-build-env-4q6h0vub\overlay' --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- setuptools wheel 'Cython>=0.28.5' 'numpy==1.13.3; python_version=='"'"'3.6'"'"' and platform_system!='"'"'AIX'"'"' and platform_python_implementation == '"'"'CPython'"'"'' 'numpy==1.14.0; python_version=='"'"'3.6'"'"' and platform_system!='"'"'AIX'"'"' and platform_python_implementation != '"'"'CPython'"'"'' 'numpy==1.14.5; python_version=='"'"'3.7'"'"' and platform_system!='"'"'AIX'"'"'' 'numpy==1.17.3; python_version>='"'"'3.8'"'"' and platform_system!='"'"'AIX'"'"'' 'numpy==1.16.0; python_version=='"'"'3.6'"'"' and platform_system=='"'"'AIX'"'"'' 'numpy==1.16.0; python_version=='"'"'3.7'"'"' and platform_system==

# Imports

In [26]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras


import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd
import pickle
import random
#nltk.download('punkt')

# Load  patterns & responses (intents.json)

In [27]:
with open('intents2.json') as file:
    data = json.load(file)
    
ws = []
cs = []
ds = []
ignore_ws = ['?']
# Check all intents
for i in data['intents']:
    # Check for pattern in each intent
    for p in i['patterns']:
        # get token from all words
        w = nltk.word_tokenize(p)
        # add to our word list
        ws.extend(w)
        ds.append((w, i['tag']))
        # add to class list
        if i['tag'] not in cs:
            cs.append(i['tag'])
# remove duplicates
ws = [stemmer.stem(w.lower()) for w in ws if w not in ignore_ws]
ws = sorted(list(set(ws)))
# sort classes
cs = sorted(list(set(cs)))
# Docs = combination between patterns and intents
print (len(ds), "docs")
# classes = intents
print (len(cs), "classes", cs)
# words = all ws, vocabulary
print (len(ws), "unique words", ws)

276 docs
11 classes ['ask_location', 'give_direction_from_A', 'give_direction_from_C', 'give_direction_from_Cube', 'give_direction_from_D', 'give_direction_from_K', 'give_direction_from_Library', 'give_direction_from_T', 'goodbye', 'greeting', 'options']
72 unique words ['!', "'s", ',', 'a', 'academ', 'adio', 'be', 'bibliotheek', 'build', 'bye', 'c', 'can', 'chat', 'ciou', 'class', 'classroom', 'cobbenh', 'could', 'cub', 'cya', 'd', 'dant', 'day', 'destin', 'do', 'find', 'for', 'get', 'good', 'goodby', 'hay', 'hello', 'help', 'hey', 'hi', 'hol', 'how', 'k', 'koopm', 'l', 'lat', 'lect', 'lectureroom', 'libr', 'loc', 'me', 'my', 'next', 'nic', 'provid', 'room', 'search', 'see', 'serv', 'show', 't', 'the', 'tia', 'til', 'tim', 'to', 'tocub', 'ub', 'ul', 'univers', 'universiteit', 'up', 'way', 'what', 'with', 'yo', 'you']


# Create train- and testdata

In [28]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(cs)
# training set, bag of words for each sentence
for doc in ds:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in ws:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[cs.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\Mels\AppData\Local\Temp/ipykernel_13028/1372038048.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [29]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [30]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200


D:\Anaconda\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


56/56 [==============================] - 0s 1ms/step - loss: 2.3283 - accuracy: 0.1812
Epoch 2/200
56/56 [==============================] - 0s 1ms/step - loss: 2.1440 - accuracy: 0.2355
Epoch 3/200
56/56 [==============================] - 0s 1ms/step - loss: 2.0008 - accuracy: 0.3152
Epoch 4/200
56/56 [==============================] - 0s 2ms/step - loss: 1.8285 - accuracy: 0.3732
Epoch 5/200
56/56 [==============================] - 0s 1ms/step - loss: 1.7411 - accuracy: 0.3804
Epoch 6/200
56/56 [==============================] - 0s 1ms/step - loss: 1.5293 - accuracy: 0.3986
Epoch 7/200
56/56 [==============================] - 0s 1ms/step - loss: 1.3974 - accuracy: 0.4783
Epoch 8/200
56/56 [==============================] - 0s 1ms/step - loss: 1.3737 - accuracy: 0.4457
Epoch 9/200
56/56 [==============================] - 0s 1ms/step - loss: 1.3453 - accuracy: 0.4094
Epoch 10/200
56/56 [==============================] - 0s 1ms/step - loss: 1.3061 - accuracy: 0.4710
Epoch 11/200
56/56 [=

56/56 [==============================] - 0s 1ms/step - loss: 0.7089 - accuracy: 0.6449
Epoch 84/200
56/56 [==============================] - 0s 1ms/step - loss: 0.7475 - accuracy: 0.5833
Epoch 85/200
56/56 [==============================] - 0s 1ms/step - loss: 0.7230 - accuracy: 0.6159
Epoch 86/200
56/56 [==============================] - 0s 1ms/step - loss: 0.6966 - accuracy: 0.6123
Epoch 87/200
56/56 [==============================] - 0s 1ms/step - loss: 0.7216 - accuracy: 0.6087
Epoch 88/200
56/56 [==============================] - 0s 2ms/step - loss: 0.6518 - accuracy: 0.6196
Epoch 89/200
56/56 [==============================] - 0s 1ms/step - loss: 0.7424 - accuracy: 0.6087
Epoch 90/200
56/56 [==============================] - 0s 1ms/step - loss: 0.6691 - accuracy: 0.6449
Epoch 91/200
56/56 [==============================] - 0s 1ms/step - loss: 0.7386 - accuracy: 0.5833
Epoch 92/200
56/56 [==============================] - 0s 1ms/step - loss: 0.7079 - accuracy: 0.5652
Epoch 93/200


56/56 [==============================] - 0s 1ms/step - loss: 0.6531 - accuracy: 0.6594
Epoch 165/200
56/56 [==============================] - 0s 2ms/step - loss: 0.6558 - accuracy: 0.6522
Epoch 166/200
56/56 [==============================] - 0s 1ms/step - loss: 0.7139 - accuracy: 0.6087
Epoch 167/200
56/56 [==============================] - 0s 1ms/step - loss: 0.6492 - accuracy: 0.6123
Epoch 168/200
56/56 [==============================] - 0s 1ms/step - loss: 0.6186 - accuracy: 0.6558
Epoch 169/200
56/56 [==============================] - 0s 1ms/step - loss: 0.6880 - accuracy: 0.6123
Epoch 170/200
56/56 [==============================] - 0s 1ms/step - loss: 0.6608 - accuracy: 0.6196
Epoch 171/200
56/56 [==============================] - 0s 1ms/step - loss: 0.6800 - accuracy: 0.5978
Epoch 172/200
56/56 [==============================] - 0s 1ms/step - loss: 0.6731 - accuracy: 0.5870
Epoch 173/200
56/56 [==============================] - 0s 1ms/step - loss: 0.6524 - accuracy: 0.6087
Epoc

In [31]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [32]:
def predict_class(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, ws)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((cs[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [40]:
def getResponse(ints, intents_json):
    tag = ints[0][0]
    print(tag)
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text)
    res = getResponse(ints,data)
    return res

In [ ]:

import tkinter
from tkinter import *

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
        
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)


#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial", background="#dddddd")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=130, y=401, height=40, width=265)
SendButton.place(x=6, y=401, height=40)

In [ ]:
base.mainloop()

# Giving real directions (unimplemented)

In [ ]:
import googlemaps
from mapbox import Directions
def calc_route(bld):
    dicbuilding = buildings.get(bld)
    if not dicbuilding is None:
        response = service.directions((global_currloc, dicbuilding),'mapbox.walking')
        print(response.geojson())
    

# API

In [ ]:
!pip install fastapi
!pip install uvicorn

from fastapi import FastAPI

app = FastAPI()

@app.get("/chat")
def hello(msg = None):

    if msg is None:
        text = 'Hello!'

    else:
        text = chatbot_response(msg)

    return text

In [56]:
pip install -U gradient

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
pytest 6.2.4 requires attrs>=19.2.0, but you have attrs 18.2.0 which is incompatible.


  Created wheel for click-completion: filename=click_completion-0.5.2-py3-none-any.whl size=11202 sha256=e20f7be7970504d11d2b6046558ac643c715810e142a0a76c42972004da0c101
  Stored in directory: c:\users\mels\appdata\local\pip\cache\wheels\f1\23\55\8287077a0e274c8d7ddf94fb623cb481cd0f4512d1a5062603
  Created wheel for gql: filename=gql-3.0.0a6-py2.py3-none-any.whl size=34865 sha256=da421df220ee12d3d8973eb3fea0466abe070a65d509f25c4790b3420d56578c
  Stored in directory: c:\users\mels\appdata\local\pip\cache\wheels\a7\29\1a\9709f3b2c98a2d811d8be839cba15af7e578815f0e8f87d96e
  Created wheel for halo: filename=halo-0.0.31-py3-none-any.whl size=11260 sha256=a95d5a2d407e25cdde3146baff2551a1f12d9d00b22802b45c30ef7380011a5f
  Stored in directory: c:\users\mels\appdata\local\pip\cache\wheels\bb\85\47\b7c7338ab52808105f937bd8c04aec5d98a543311ac2c8bed2
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=2c7a45384d96cced9d5dd58259779f0589a975a86ec4e476

In [58]:
gradient apiKey c3cc0098cd4fd2918bf10d9ce291b4

SyntaxError: invalid syntax (Temp/ipykernel_13028/1037572133.py, line 1)